In [16]:
import os
import json
import numpy as np
import tensorflow as tf

In [17]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [18]:
# Name of directory for grids
directory_name = './data/small_grids/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        y_train.append(i['direction'])
    
    # Close file socket
    f.close()

In [19]:
# Name of directory for grids
directory_name = './data/small_grids_test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [20]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 30, 30) ) / 9
train_in_position = np.reshape( x_train_position, (-1, 2, 1) ) / 9
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 30, 30) ) / 9
test_in_position = np.reshape( x_test_position, (-1, 2, 1) ) / 9
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [22]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (30,30) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [23]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (2,1) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [24]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.concatenate([flatten_grid, flatten_position], axis=1)

In [36]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 50, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 10, activation = tf.nn.relu )( dense_2 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_3 )
probabilities = tf.keras.layers.Softmax()( logits )

In [37]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, position_input], outputs = probabilities )
model.compile( optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [38]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    predictions = np.argmax( model.predict( data ), axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [42]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_position], y_test )

0	0	2	3
0	0	7	10
0	0	18	24
0	0	13	51


In [45]:
# Train the model
history = model.fit( [train_in_grid, train_in_position], train_out, epochs = 20 )

Epoch 1/20
7/7 [==============================] - 0s 9ms/step - loss: 0.9680 - accuracy: 0.6085
Epoch 2/20
7/7 [==============================] - 0s 14ms/step - loss: 0.9664 - accuracy: 0.5943
Epoch 3/20
7/7 [==============================] - 0s 6ms/step - loss: 0.9628 - accuracy: 0.5849
Epoch 4/20
7/7 [==============================] - 0s 6ms/step - loss: 0.9700 - accuracy: 0.5943
Epoch 5/20
7/7 [==============================] - 0s 5ms/step - loss: 0.9830 - accuracy: 0.6132
Epoch 6/20
7/7 [==============================] - 0s 5ms/step - loss: 0.9717 - accuracy: 0.5802
Epoch 7/20
7/7 [==============================] - 0s 5ms/step - loss: 0.9591 - accuracy: 0.6038
Epoch 8/20
7/7 [==============================] - 0s 3ms/step - loss: 0.9590 - accuracy: 0.5849
Epoch 9/20
7/7 [==============================] - 0s 5ms/step - loss: 0.9629 - accuracy: 0.6085
Epoch 10/20
7/7 [==============================] - 0s 4ms/step - loss: 0.9631 - accuracy: 0.6085
Epoch 11/20
7/7 [=====================

In [46]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_position], y_test )

0	0	0	5
0	0	0	17
0	0	0	42
0	0	0	64
